In [2]:
import pandas as pd

In [3]:
music_data = pd.read_csv('Programas/Music_R_S/Music Data/tracks_features.csv')


In [4]:
print(music_data.shape)
print(music_data.size)

(1204025, 24)
28896600


Redimentionate the dataset

In [5]:
#here i reduce the dimentionality to 15,000 because my laptop is shit
data = music_data.sample(n=1000, random_state=0)
#here i just fix the indexes so they are from 1 - 15,000
data = data.reset_index(drop= True)
data.index = data.index + 1

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 1 to 1000
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1000 non-null   object 
 1   name              1000 non-null   object 
 2   album             1000 non-null   object 
 3   album_id          1000 non-null   object 
 4   artists           1000 non-null   object 
 5   artist_ids        1000 non-null   object 
 6   track_number      1000 non-null   int64  
 7   disc_number       1000 non-null   int64  
 8   explicit          1000 non-null   bool   
 9   danceability      1000 non-null   float64
 10  energy            1000 non-null   float64
 11  key               1000 non-null   int64  
 12  loudness          1000 non-null   float64
 13  mode              1000 non-null   int64  
 14  speechiness       1000 non-null   float64
 15  acousticness      1000 non-null   float64
 16  instrumentalness  1000 non-null   float64


Data cleaning

In [7]:
#drop useless data
data.drop(['album_id', 'artist_ids', 'track_number', 'disc_number', 'time_signature', 'release_date'],axis= 1,  inplace= True)

In [8]:
#unite all "musical" features in one
data['song_features'] = data.apply(lambda row: ', '.join([str(row[col]) for col in data.columns [2:]]), axis= 1)

In [9]:
data.columns

Index(['id', 'name', 'album', 'artists', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'year', 'song_features'],
      dtype='object')

In [10]:
#drop more data
data.drop(['album', 'artists', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'year'], axis= 1, inplace= True)

In [11]:
data.head()

,id,name,song_features
1,1T1RyfNzTKWaLQuLiwWdjH,Sunday School,"Redemption, ['The Loyalists'], False, 0.532, 0..."
2,3cpJbarmU5e9jL02XsOkkv,Moments Before Dawn,"Azure Pieces of Life, ['Kat Epple', 'Chuck Gri..."
3,1l9YCCXXfVxy0aBXFcjTj5,Things Ain't What They Used To Be,"Lush Life - A Musical Journey, ['Joe Castro'],..."
4,0RjEvsCxOxoC5tcIdOdpV6,F**kin' Perfect,"Piano Tribute to P!nk, ['Piano Tribute Players..."
5,3K0r4oPoNucAvwJIfFVYvg,Here For You,"Release Some Tension, ['SWV'], False, 0.6, 0.5..."


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 1 to 1000
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1000 non-null   object
 1   name           1000 non-null   object
 2   song_features  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [13]:
#new dataset
music = data[['id', 'name', 'song_features']]

In [14]:
music.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 1 to 1000
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1000 non-null   object
 1   name           1000 non-null   object
 2   song_features  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


Vectorization

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
#put the CV in a variable and fix the max features to the data level and erease the stop words
cv = CountVectorizer(max_features= 15000, stop_words= 'english')

In [17]:
cv

CountVectorizer(max_features=15000, stop_words='english')

In [18]:
#here we vectorize all the song features in an unicode
vectors = cv.fit_transform(music['song_features'].values.astype('U')).toarray()

In [19]:
vectors.shape

(1000, 7014)

Cosine similarity

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
#use the cosine similarity and use it with the vectors
similarity = cosine_similarity(vectors)

In [22]:
similarity

array([[1.        , 0.05455447, 0.0559017 , ..., 0.04166667, 0.05892557,
        0.        ],
       [0.05455447, 1.        , 0.09759001, ..., 0.03636965, 0.05143445,
        0.05455447],
       [0.0559017 , 0.09759001, 1.        , ..., 0.0372678 , 0.10540926,
        0.0559017 ],
       ...,
       [0.04166667, 0.03636965, 0.0372678 , ..., 1.        , 0.03928371,
        0.        ],
       [0.05892557, 0.05143445, 0.10540926, ..., 0.03928371, 1.        ,
        0.        ],
       [0.        , 0.05455447, 0.0559017 , ..., 0.        , 0.        ,
        1.        ]])

In [23]:
music.head(3)

,id,name,song_features
1,1T1RyfNzTKWaLQuLiwWdjH,Sunday School,"Redemption, ['The Loyalists'], False, 0.532, 0..."
2,3cpJbarmU5e9jL02XsOkkv,Moments Before Dawn,"Azure Pieces of Life, ['Kat Epple', 'Chuck Gri..."
3,1l9YCCXXfVxy0aBXFcjTj5,Things Ain't What They Used To Be,"Lush Life - A Musical Journey, ['Joe Castro'],..."


In [24]:
#first taste of the program working
distance = sorted(enumerate(similarity[0]), reverse=True, key=lambda x: x[1])
for i in distance[:5]:
  print(music.iloc[i[0]]['name'])

Sunday School
Downtown Mayors Goodnight, Alley Kids Rule! [S-2 Beatdown Mayors Mix]
Meridian Leeward
Get Free
Go to Sleep


In [33]:
music[music['name'] == 'Girl On the Moon']

,id,name,song_features
155,4M8pelLdIxhQvlMTjiseig,Girl On the Moon,"Sad Rock Songs, ['Foreigner'], False, 0.597, 0..."


In [26]:
#function to make the program work
def recommend(data):
    index = music[music['name'] == data].index[0]
    distance = sorted(enumerate(similarity[index]), reverse=True, key=lambda x: x[1])
    for i in distance[0:6]:
      print(music.iloc[i[0]]['name']) 

In [34]:
#this is the model working, with random
import random
recommend(random.choice(music['name']))

Palm Leaf (arr. G. de Chiaro)
Minuet In Jazz
Pieces de clavecin, Book 1: La Boullongne
Sally in Our Alley
Over the Mountain
Six Disciples


Save the model

In [61]:
import pickle

In [90]:
#save the data
data_path = 'Programas/Music_R_S/songs_data.pkl'
with open(data_path, 'wb') as music_file:
    pickle.dump(music_data, music_file)

In [91]:
#save the "model" similarity
model_path = 'Programas/Music_R_S/similarity.pkl'
with open(model_path, 'wb') as model_file:
    pickle.dump(similarity, model_file)

In [102]:
pickle.load(open('songs_data.pkl', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'songs_data.pkl'

In [94]:
music.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 1 to 1000
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1000 non-null   object
 1   name           1000 non-null   object
 2   song_features  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB
